In [19]:
import os
import urllib
import shutil
import azureml

from azureml.core import Experiment
from azureml.core import Workspace, Run

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.dnn import TensorFlow

In [1]:
!pip install --upgrade pip

     |████████████████████████████████| 1.5 MB 7.7 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 20.0.2
    Uninstalling pip-20.0.2:
      Successfully uninstalled pip-20.0.2


In [2]:
!pip install --upgrade tensorflow==2.2.0

     |█████████████████████           | 338.3 MB 91.9 MB/s eta 0:00:024     |███████▊                        | 125.0 MB 110.9 MB/s eta 0:00:04     |███████████████                 | 242.4 MB 96.2 MB/s eta 0:00:03

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 516.2 MB 7.0 kB/s s eta 0:00:01��█████████████████████  | 483.7 MB 108.4 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 42.4 MB/s eta 0:00:01
     |████████████████████████████████| 454 kB 45.4 MB/s eta 0:00:01
ERROR: tensorflow-gpu 2.1.0 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
ERROR: tensorflow-gpu 2.1.0 has requirement tensorboard<2.2.0,>=2.1.0, but you'll have tensorboard 2.2.1 which is incompatible.
ERROR: tensorflow-gpu 2.1.0 has requirement tensorflow-estimator<2.2.0,>=2.1.0rc0, but you'll have tensorflow-estimator 2.2.0 which is incompatible.
  Attempting uninstall: h5py
    Found existing installation: h5py 2.9.0
    Uninstalling h5py-2.9.0:
      Successfully uninstalled h5py-2.9.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.1.0
    Uninstalling tensorflow-estimator-2.1.0:
      Successfully uninstalled tensorflow-estimator-2.

In [16]:
!pip install azureml.logging

ERROR: Could not find a version that satisfies the requirement azureml.logging (from versions: none)
ERROR: No matching distribution found for azureml.logging


In [20]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [21]:
ws = Workspace.from_config()

In [22]:
ws

Workspace.create(name='gputraining', subscription_id='c46a9435-c957-4e6c-a0f4-b9a597984773', resource_group='mlops')

In [23]:
script_folder = './tf-mnist'
os.makedirs(script_folder, exist_ok=True)

exp = Experiment(workspace=ws, name='tf-mnist')

In [24]:
from azureml.core.dataset import Dataset

web_paths = [
            'http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz',
            'http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz',
            'http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
            'http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz'
            ]
dataset = Dataset.File.from_files(path=web_paths)

In [25]:
dataset = dataset.register(workspace=ws,
                           name='mnist dataset',
                           description='training and test dataset',
                           create_new_version=True)

# list the files referenced by dataset
dataset.to_path()

['/http/yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz',
 '/http/yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz',
 '/http/yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
 '/http/yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz']

In [26]:
cluster_name = "gpucluster1"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

Found existing compute target


In [27]:
script_params = {
    '--data-folder': dataset.as_named_input('mnist').as_mount(),
    '--batch-size': 50,
    '--first-layer-neurons': 300,
    '--second-layer-neurons': 100,
    '--learning-rate': 0.01
}

est = TensorFlow(source_directory=script_folder,
                 entry_script='tf_mnist.py',
                 script_params=script_params,
                 compute_target=compute_target,
                 use_gpu=True,
                 pip_packages=['azureml-dataprep[pandas,fuse]'])

WARNING - framework_version is not specified, defaulting to version 1.13.
WARNING - You have specified to install packages in your run. Note that you have overridden Azure ML's installation of the following packages: ['azureml-dataprep']. We cannot guarantee image build will succeed.


In [30]:
run = exp.submit(est)
run.wait_for_completion(show_output=True)

RunId: tf-mnist_1594515227_43c16e73
Web View: https://ml.azure.com/experiments/tf-mnist/runs/tf-mnist_1594515227_43c16e73?wsid=/subscriptions/c46a9435-c957-4e6c-a0f4-b9a597984773/resourcegroups/mlops/workspaces/gputraining

Streaming azureml-logs/55_azureml-execution-tvmps_c4bca20bd952bede855b6ce84989bd025280fbbc42ced880e9fd39a96792d523_d.txt

2020-07-12T00:54:08Z Executing 'Copy ACR Details file' on 10.0.0.4
2020-07-12T00:54:08Z Starting output-watcher...
2020-07-12T00:54:09Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
2020-07-12T00:54:09Z Copy ACR Details file succeeded on 10.0.0.4. Output: 
>>>   
>>>   
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_1b21012d99b68bea171596e10bcaefa4
f7277927d38a: Already exists
8d3eac894db4: Already exists
edf72af6d627: Already exists
3e4f86211d23: Already exists
d6e9603ff777: Already exists
5cad422780e2: Already exists
8130687c8acb: Already exists
c11e9246d621: Already exists
0dfae24cbbd9: Already e

{'runId': 'tf-mnist_1594515227_43c16e73',
 'target': 'gpucluster1',
 'status': 'Completed',
 'startTimeUtc': '2020-07-12T00:54:11.090638Z',
 'endTimeUtc': '2020-07-12T00:56:27.649946Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '2d33e7cc-ace5-44d1-bef2-dd84f6fa206b',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': '7b573201-f261-434b-bb36-a102a6dcfe45'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'mnist', 'mechanism': 'Mount'}}],
 'runDefinition': {'script': 'tf_mnist.py',
  'useAbsolutePath': False,
  'arguments': ['--data-folder',
   'DatasetConsumptionConfig:mnist',
   '--batch-size',
   '50',
   '--first-layer-neurons',
   '300',
   '--second-layer-neurons',
   '100',
   '--learning-rate',
   '0.01'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'gpucluster1',
  'dataRefer

In [35]:
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

model = run.register_model(model_name='tf-dnn-mnist', 
                           model_path='outputs/model',
                           model_framework=Model.Framework.TENSORFLOW,
                           model_framework_version='1.13.0',
                           resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5))

ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path outputs/model in the set of files uploaded to the run: ['azureml-logs/55_azureml-execution-tvmps_c4bca20bd952bede855b6ce84989bd025280fbbc42ced880e9fd39a96792d523_d.txt', 'azureml-logs/65_job_prep-tvmps_c4bca20bd952bede855b6ce84989bd025280fbbc42ced880e9fd39a96792d523_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_c4bca20bd952bede855b6ce84989bd025280fbbc42ced880e9fd39a96792d523_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/121_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log']
                See https://aka.ms/run-logging for more details.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Could not locate the provided model_path outputs/model in the set of files uploaded to the run: ['azureml-logs/55_azureml-execution-tvmps_c4bca20bd952bede855b6ce

In [32]:
# Create a model folder in the current directory
os.makedirs('./model', exist_ok=True)

for f in run.get_file_names():
    if f.startswith('outputs/model'):
        output_file_path = os.path.join('./model', f.split('/')[-1])
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        run.download_file(name=f, output_file_path=output_file_path)

In [ ]:
from azureml.core.runconfig import MpiConfiguration
from azureml.train.dnn import TensorFlow

# Tensorflow constructor
estimator= TensorFlow(source_directory=project_folder,
                      compute_target=compute_target,
                      script_params=script_params,
                      entry_script='script.py',
                      node_count=2,
                      process_count_per_node=1,
                      distributed_training=MpiConfiguration(),
                      framework_version='1.13',
                      use_gpu=True,
                      pip_packages=['azureml-dataprep[pandas,fuse]'])

In [ ]:
from azureml.train.dnn import TensorFlow

distributed_training = TensorflowConfiguration()
distributed_training.worker_count = 2

# Tensorflow constructor
tf_est= TensorFlow(source_directory=project_folder,
                      compute_target=compute_target,
                      script_params=script_params,
                      entry_script='script.py',
                      node_count=2,
                      process_count_per_node=1,
                      distributed_training=distributed_training,
                      use_gpu=True,
                      pip_packages=['azureml-dataprep[pandas,fuse]'])

# submit the TensorFlow job
run = exp.submit(tf_est)

In [ ]:
TF_CONFIG='{
    "cluster": {
        "ps": ["host0:2222", "host1:2222"],
        "worker": ["host2:2222", "host3:2222", "host4:2222"],
    },
    "task": {"type": "ps", "index": 0},
    "environment": "cloud"
}'

In [ ]:
import os, json
import tensorflow as tf

tf_config = os.environ.get('TF_CONFIG')
if not tf_config or tf_config == "":
    raise ValueError("TF_CONFIG not found.")
tf_config_json = json.loads(tf_config)
cluster_spec = tf.train.ClusterSpec(cluster)

In [ ]:
service = Model.deploy(ws, "tensorflow-web-service", [model])